## Bonusový úkol č. 2 - stahování dat z webového zdroje
Vytvořte funkci **sync()**, která získá kompletní seznam produktů (tj. včetně dalších stránek) dostupných v kategorii
https://www.alza.cz/bezzrcadlovky-bez-objektivu/18863907.htm
a u každého produktu zjistí jeho aktuální cenu a stav skladu.
Funkce bude uchovávat získané informace a historii změn v relační databázi SQLLite3 obsahující dvě tabulky:  
* tabulku `products` a  
* tabulku `products_history`.

Struktura obou tabulek je shodná a obsahuje následující sloupce:  
* `id` TEXT - id produktu, např. OS072i1l1 (viz data-impression-id),  
* `url` TEXT - url produktu k kterému se vztahuje cena (pouze část path, viz ukázka na konci),  
* `title` TEXT - název produktu,  
* `price` DECIMAL - cena produktu s DPH k danému datu,   
* `stock_state` TEXT - stav skladu k danému datu,  
* `last_update` DATETIME - datum poslední změny hodnot záznamu v UTC  

Do tabulky `products_history` zkopírujte záznam z tabulky `products` ve chvíli, kdy se změnil nějaký sledovaný údaj (název, cena nebo stav skladu) a je potřeba aktualizovat data v tabulce `products`. Pozor, jedno `id` může mít více variant `url` s různou cenou. Při opětovném volání funkce **sync()** se prověří existence záznamu v `products`, prověří se shoda hodnot a vždy aktualizuje hodnota `last_update`, aby bylo zřejmé, ke kterému datu je informace platná.

**Předpokládaná náročnost**: 1 hodina

### Závislosti, načtení knihoven

V následující buňce deklarujte všechny závislosti

In [1]:
%pip install requests requests_cache bs4

import requests, requests_cache, sqlite3, random, datetime
from bs4 import BeautifulSoup

#pro vývoj je vhodné zapnout cache (viz přednáška), pro finalní otestovaní tento řádek zakomentujte
#requests_cache.install_cache('devel') 

#nadeklarujeme si novy typ sloupce DECIMAL do sqlite3, abychom měli automatický převod mezi SQLite3 a Python
from decimal import Decimal
sqlite3.register_adapter(Decimal, lambda d: str(d))
sqlite3.register_converter("DECIMAL", lambda s: Decimal(s.decode('ascii')))

Note: you may need to restart the kernel to use updated packages.


### Deklarace funkce

V následujícím boxu definujte funkci **sync(name)** s jedním parametrem (název souboru s DB), která provede zadanou operaci. 
Pro přístup k DB lze s ohledem na složitost zadání použít přímo funkcionalitu vestavěného modulu sqlite3 (viz https://docs.python.org/2/library/sqlite3.html).

**TIP**: pro získání seznamu všech produktů lze použít endpoint https://www.alza.cz/Services/EShopService.svc/Filter

Mohlo by se také hodit: https://curl.trillworks.com/

In [4]:
# V tomto boxu pouze implementujte funkci ale nevolejte ji (pro vývoj si vytvořte vlastní buňky).
# nezapomeňte na cookies a hlavičky, jinak se Vám může zobrazit otázka "nejste robot?"
def sync(dbfile='data.sqlite'):
    with sqlite3.connect(dbfile, detect_types=sqlite3.PARSE_DECLTYPES) as conn:
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS products
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME, PRIMARY KEY(id,url))''')
        
        c.execute('''CREATE TABLE IF NOT EXISTS products_history
                  (id TEXT, url TEXT, title TEXT, price DECIMAL, stock_state TEXT, last_update DATETIME)''')

        c.execute('''CREATE INDEX IF NOT EXISTS idx_id ON products (id)''')
        c.execute('''CREATE INDEX IF NOT EXISTS idx_idurl ON products_history (id, url)''')

        s = requests.session()
        
        #Get data from alza using filter
        resp = requests.post('https://www.alza.cz/Services/EShopService.svc/Filter', json={
                'idCategory': 18863907, 
                'producers': '', 
                'parameters': [], 
                'idPrefix': 0, 
                'prefixType': 0, 
                'page': 1, 
                'pageTo': 4, 
                'inStock': False, 
                'newsOnly': False, 
                'commodityStatusType': None, 
                'upperDescriptionStatus': 0, 
                'branchId': -2, 
                'sort': 0, 
                'categoryType': 1, 
                'searchTerm': '', 
                'sendProducers': False, 
                'layout': 0, 
                'append': False, 
                'leasingCatId': None, 
                'yearFrom': None, 
                'yearTo': None, 
                'artistId': None, 
                'minPrice': -1, 
                'maxPrice': -1, 
                'shouldDisplayVirtooal': False, 
                'callFromParametrizationDialog': False, 
                'commodityWearType': None, 
                'scroll': 10905, 
                'hash': '#f&cst=null&cud=0&pg=1-4&prod=', 
                'counter': 3
                }, headers = {
            'Content-Type': 'application/json'},)

        #parse json, then parse html, save all necessary info
        data = resp.json()
        data_html = data['d']['Boxes']
        beautiful_html = BeautifulSoup(data_html, 'html.parser')
        elements =  beautiful_html.find_all('a', {'class':'name browsinglink'})
        for element in elements:
            id = element.get("data-impression-id")
            url = element.get("href")
            title = element.get("data-impression-name")
            price = element.get("data-impression-metric2").replace(',', '.')
            stock_state = element.get("data-impression-dimension13").replace('\xa0','')
            last_update = datetime.datetime.utcnow()

            #try to find if something changed in DB
            c.execute("SELECT id, url, title, price, stock_state FROM products WHERE id=? AND url=? AND title=? AND price=? AND stock_state=?",(id, url, title, price, stock_state))
            result = c.fetchone()

            #if nothing changed, update time in DB
            if result:
                c.execute("UPDATE products SET last_update=? WHERE id=? AND url=?", (last_update, id, url))
            # else if just one value changed, place info about changed product into products_history table and replace old values with new ones
            else:
                c.execute("INSERT INTO products_history SELECT * FROM products WHERE id=? AND url=?", (id, url))
                c.execute("REPLACE INTO products VALUES (?, ?, ?, ?, ?, ?)", (id, url, title, price, stock_state, last_update))
                
        conn.commit()        
        c.close()

### Ověření korektní funkce

Na následujícím kódu lze ověřit základní funkcionalitu. Měly byste dostat stejný výstup jako je v ukázce. Protože se však stav e-shopu může měnit, uzpůsobte si eventuelně dotaz dle potřeb. Momentálně se testuje existence produktu https://www.alza.cz/sony-alpha-7ii?dq=2286288 ev. 
https://www.alza.cz/kod/OS072i1p5.

Při ověřování korektní funkce Vaší implementace bude porovnán obsah DB vytvořený Vaší funkcí s předpokládaným obsahem DB v určitou dobu a poté znovu s několika hodinovým odstupem.

In [5]:
from contextlib import closing

sync('data.sqlite')

with sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_DECLTYPES) as conn:
    with closing(conn.cursor()) as c:
        c.execute('SELECT id, url, price FROM products WHERE id=? AND url=? AND price>20000', ('OS072i1p5','/sony-alpha-7ii?dq=2286288'))
        r = c.fetchone()
        print(r)
        assert(r != None)

        c.execute('SELECT id, url, price FROM products WHERE id=? AND price>30000', ('OF7032a',))
        r = c.fetchall()
        print(r)
        assert (len(r)>0 and '/fujifilm-x-t3?dq=5457426' in [a[1] for a in r])


print("OK")        

('OS072i1p5', '/sony-alpha-7ii?dq=2286288', Decimal('31490'))
[('OF7032a', '/fujifilm-x-t3?dq=5457426', Decimal('39990')), ('OF7032a', '/fujifilm-x-t3-telo-cerny-levne-d5754350.htm', Decimal('36989.7')), ('OF7032a', '/fujifilm-x-t3-telo-cerny-sleva-d5877920.htm', Decimal('33990.11'))]
OK


### Komentář
Do pole níže můžete vložit textový komentář týkající se tohoto úkolu. Např. jak dlouho Vám trvalo řešení, co bylo obtížné, co bylo se mělo více v rámci přenášky vysvětlit apod.

n/a